# mCT DL-HMC Training & Testing template - baseline V1
Works for both single and multi subjects studies. <br>
Last update (01/23): first version, DLHMC baseline, use point cloud
resize 96*96*64

## Which function need modifications?
Every function that needs to use google shared document service. We may need a new shared sheet for mCT. get_data, print_info, compute_delta_T.


In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    Spacingd, \
    ToTensord,  \
    DataStatsd, \
    ToDeviced
from monai.data import list_data_collate
import torch
import pytorch_lightning as pl
from torchsummary import summary
monai.config.print_config()
import sys
sys.path.append(r'../util/python')
import vicra_toolbox

2023-01-26 16:56:57,540 - Created a temporary directory at /tmp/tmppqz66r45
2023-01-26 16:56:57,542 - Writing /tmp/tmppqz66r45/_remote_module_non_scriptable.py
MONAI version: 1.0.1
Numpy version: 1.23.4
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8271a193229fe4437026185e218d5b06f7c8ce69
MONAI __file__: /home1/el754/anaconda3/envs/dl-hmc_2301/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.11.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.14.1
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UN

## Data loading and sampling

In [2]:
# Load available data
#!!! Function get_data & print_info needs update

from mCTdata_summary_toolbox import get_data, print_info

df, df_ready, patients_ready =get_data('dl-hmc-data-analysis-08b527b43bc1.json', ['FPEB'])
print_info('dl-hmc-data-analysis-08b527b43bc1.json', ['FPEB'])

For tracer  FPEB   1  patients out of  1  are ready.
Ready patients are ['EC878']


In [3]:
''' You can also get and visualize information using other functions of the dataset_summary toolbox, such as

-compute_delta_T: compute delta_T for each patient and add it to the summary
-plot_delta_T_all: plot the delta_T computed by compute_delta_T
-delta_T_norm: normalizes delta_T and plots it
-delta_T_smoothed: smoothes delta_T_norm and plots it
-data_analysis_display: displays information about the motion (magnitude, when do the major movements occur, mean and SD of delta_T...)
-average_motion: computes and plot the average delta_T at each second among patients
-make_gif_3Dcloud: make a gif with the 3D cloud data for a list of patients '''

' You can also get and visualize information using other functions of the dataset_summary toolbox, such as\n\n-compute_delta_T: compute delta_T for each patient and add it to the summary\n-plot_delta_T_all: plot the delta_T computed by compute_delta_T\n-delta_T_norm: normalizes delta_T and plots it\n-delta_T_smoothed: smoothes delta_T_norm and plots it\n-data_analysis_display: displays information about the motion (magnitude, when do the major movements occur, mean and SD of delta_T...)\n-average_motion: computes and plot the average delta_T at each second among patients\n-make_gif_3Dcloud: make a gif with the 3D cloud data for a list of patients '

In [4]:
df_ready

,PatientID,Name,InjectionID,date,injtime,dose,tracer,isotope,protocol,Diagnosis,Secondary_Diagnosis,MotMag_entire_EM_mm,Status (Done/Running),MCCOD QA,Training/Testing_TZ
0,EC878,"EC878, human",27647,2021/6/30,11:36:52,test,FPEB,F-18,fpeb_sam,Healthy Control,,,Done,,Training


In [5]:
# Choose training and testing patients

tr_set=['EC878']
test_set=['EC878']

In [6]:
inters=set(tr_set) & set(test_set)
if len(inters)>0:
    print('Careful, there\'s data contamination between the training and testing sets. The duplicates are: ', inters)

Careful, there's data contamination between the training and testing sets. The duplicates are:  {'EC878'}


In [7]:
# Split and sample the training set

from mCTdata_summary_toolbox import compute_delta_T #!!!needs modification
from data_prep_toolbox import delta_T_magnitude, Relative_motion_A_to_B_12, build_legal_dataset, deal_dataframe, clean_df
from sampling_toolbox import data_split_sample, add_T_deltaT

# Get the summaries

summaries, delta_T_all=compute_delta_T(['FPEB'], tr_set)

In [8]:
summaries[0]

,PatientID,InjectionID,PatientType,Listmode,ScanStartTime,InjectionTime,ScanStartTime_ms,InjectionTime_ms,ScanStart,ScanEnd,...,VC_31,VC_32,VC_33,VC_34,Total_MC_NAC_MOLAR,MATRIX,T,delta_T,relative_T,delta_relative_T
0,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,3600,3601,...,-0.004880,0.007455,0.999960,0.724491,/data16/public/registration-brain_PET/data/mCT...,"[0.999362357638429, -0.0353334906311168, 0.005...","[-4.46406125, 2.53822725, 0.724491071428571, 0...",0.000000,"[-8.881784197001252e-16, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000
1,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,3601,3602,...,-0.004754,0.009047,0.999948,0.608685,/data16/public/registration-brain_PET/data/mCT...,"[0.99934443868892, -0.0358460208699417, 0.0050...","[-4.50590364516129, 2.6220414516129, 0.6086852...",0.031402,"[-0.03828606299244708, 0.08437468462738718, -0...",0.031430
2,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,3602,3603,...,-0.004433,0.008865,0.999951,0.609044,/data16/public/registration-brain_PET/data/mCT...,"[0.999343615855855, -0.0359136173031574, 0.004...","[-4.4401476, 2.59394656666667, 0.6090442666666...",0.025259,"[0.02643257376442154, 0.05397740980991328, -0....",0.025287
3,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,3603,3604,...,-0.003637,0.008780,0.999955,0.655729,/data16/public/registration-brain_PET/data/mCT...,"[0.999341876073333, -0.036058315463197, 0.0039...","[-4.30128575862069, 2.53551010344828, 0.655728...",0.043720,"[0.16291113868800622, -0.008979398107225922, -...",0.043753
4,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,3604,3605,...,-0.003526,0.009111,0.999952,0.231922,/data16/public/registration-brain_PET/data/mCT...,"[0.999386936346206, -0.0347994224440988, 0.003...","[-4.25216721428571, 2.6534025, 0.2319224642857...",0.316811,"[0.2182363543998953, 0.10394125995918646, -0.4...",0.316781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,5395,5396,...,-0.004627,0.018429,0.999819,1.285900,/data16/public/registration-brain_PET/data/mCT...,"[0.999272461826772, -0.0377655835393347, 0.005...","[-3.10054367857143, 4.13514392857143, 1.285900...",5.138749,"[1.416392163174896, 1.5518823878026469, 0.5567...",5.139684
1796,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,5396,5397,...,-0.003272,0.019591,0.999803,1.301960,/data16/public/registration-brain_PET/data/mCT...,"[0.999233385389177, -0.0389406495462495, 0.004...","[-2.85397375862069, 4.15582465517241, 1.301959...",6.075653,"[1.66345797910851, 1.5639571662133216, 0.57395...",6.077201
1797,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,5397,5398,...,-0.004076,0.019109,0.999809,1.153582,/data16/public/registration-brain_PET/data/mCT...,"[0.999250743143787, -0.0384036533345356, 0.004...","[-3.15098257142857, 4.24474357142857, 1.153581...",5.298290,"[1.3705077050119137, 1.6622063427765834, 0.423...",5.299549
1798,EC878,27647,Real,/data4/mct/raw/20210630_EC878/EC878.PT.PET_07_...,11:36:46,11:36:52,41806000,41812000,5398,5399,...,-0.003654,0.015067,0.999880,0.862344,/data16/public/registration-brain_PET/data/mCT...,"[0.999289813729148, -0.0374444897908747, 0.004...","[-3.00828286666667, 4.1924976, 0.8623435666666...",5.084119,"[1.5126896405816925, 1.6027812982151257, 0.133...",5.084679


In [9]:
# Sample the data, build the reference and moving images pairs 

df_train_all=[]
df_val_all=[]
df_test_all=[]

for df in summaries:
    df = clean_df(df)
    df_train_tmp, df_val_tmp, df_test_tmp=data_split_sample(df, train_frac=2.0, val_frac=0.1, test_frac=0.1, ref_suffix="_ref", mov_suffix="_mov", force_first_frame=True)
    df_train, df_val, df_test=add_T_deltaT(df_train_tmp, df_val_tmp, df_test_tmp)
    
    df_train_all.append(df_train)
    df_val_all.append(df_val)
    df_test_all.append(df_test)
    
# # Concatenate the sets for the different patients
    
df_train=pd.concat(df_train_all).reset_index(drop=True)
df_val=pd.concat(df_val_all).reset_index(drop=True)
df_test=pd.concat(df_test_all).reset_index(drop=True)

In [10]:
df_train

,PatientID_ref,InjectionID_ref,PatientType_ref,ScanStart_ref,ThreeD_Cloud_ref,COD_nomask_ref,M_ref,T_ref,PatientID_mov,InjectionID_mov,PatientType_mov,ScanStart_mov,ThreeD_Cloud_mov,COD_nomask_mov,M_mov,T_mov,delta_t,T
0,EC878,27647,Real,3600,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999362357638429, -0.0353334906311168, 0.005...","[-4.46406125, 2.53822725, 0.724491071428571, 0...",EC878,27647,Real,3600,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999362357638429, -0.0353334906311168, 0.005...","[-4.46406125, 2.53822725, 0.724491071428571, 0...",0,"[-8.881784197001252e-16, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,EC878,27647,Real,3601,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99934443868892, -0.0358460208699417, 0.0050...","[-4.50590364516129, 2.6220414516129, 0.6086852...",EC878,27647,Real,3739,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999105755253911, -0.041910355388453, 0.0055...","[-5.26997489655172, 2.56319865517241, 0.845000...",138,"[-0.7668055631451347, -0.02927560379367744, 0...."
2,EC878,27647,Real,3601,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99934443868892, -0.0358460208699417, 0.0050...","[-4.50590364516129, 2.6220414516129, 0.6086852...",EC878,27647,Real,3888,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999191558518914, -0.039751718703837, 0.0060...","[-5.16399192592593, 2.46367407407407, 0.683560...",287,"[-0.6636966017294661, -0.133991836374312, 0.07..."
3,EC878,27647,Real,3601,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99934443868892, -0.0358460208699417, 0.0050...","[-4.50590364516129, 2.6220414516129, 0.6086852...",EC878,27647,Real,4020,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999265896432288, -0.0379233360272647, 0.005...","[-4.75246571428571, 2.99214746428571, 0.082306...",419,"[-0.23061514101659952, 0.37392888303164984, -0..."
4,EC878,27647,Real,3601,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99934443868892, -0.0358460208699417, 0.0050...","[-4.50590364516129, 2.6220414516129, 0.6086852...",EC878,27647,Real,5337,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99929751544167, -0.0363997504826481, 0.0089...","[-4.5437501, 3.51403503333333, 0.9195492333333...",1736,"[-0.007285641099425533, 0.895552956548515, 0.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,EC878,27647,Real,5310,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999306359855254, -0.0362625060505174, 0.008...","[-4.31017620689655, 3.44430251724138, 1.127048...",EC878,27647,Real,5387,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999185261855515, -0.0402807621062096, 0.002...","[-3.4020986, 3.87606696666667, 1.8187529666666...",77,"[0.917657643191752, 0.4086269358526362, 0.6931..."
3597,EC878,27647,Real,5320,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999336719151401, -0.0356190613522058, 0.007...","[-4.03742644827586, 3.40788393103448, 0.505687...",EC878,27647,Real,5328,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.999318992433449, -0.0358520941018559, 0.008...","[-4.34114962068965, 3.33717231034483, 1.234143...",8,"[-0.3113114768354679, -0.05264947240864881, 0...."
3598,EC878,27647,Real,5332,/data16/public/registration-brain_PET/data/mCT...,/data16/public/registration-brain_PET/data/mCT...,"[0.99

In [11]:
# df_train.to_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_train.csv', index=False)
# df_val.to_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_val.csv', index=False)
# df_test.to_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_test.csv', index=False)

In [12]:
# Load existing sampled data

df_train=pd.read_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_train.csv')
df_val=pd.read_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_val.csv')
df_test=pd.read_csv('/data16/public/registration-brain_PET/sampling/eleonore/21mulsub_baseline/df_test.csv')

In [13]:
df_test

,PatientID_ref,InjectionID_ref,PatientType_ref,ScanStart_ref,ThreeD_Cloud_ref,COD_nomask_ref,M_ref,T_ref,PatientID_mov,InjectionID_mov,PatientType_mov,ScanStart_mov,ThreeD_Cloud_mov,COD_nomask_mov,M_mov,T_mov,delta_t,T
0,AH017,18964,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936577870891, -0.0112608471333296, 0.000...",[-1.22969731 1.25002269 2.86874875 0.650720...,AH017,18964,Real,3601,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936720854813, -0.0112424723355348, 0.000...",[-1.1975015 1.25376206 2.90297559 0.657064...,1,[ 0.03223399 0.0037651 0.03418807 0.006484...
1,AH017,18964,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936577870891, -0.0112608471333296, 0.000...",[-1.22969731 1.25002269 2.86874875 0.650720...,AH017,18964,Real,3603,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936584555769, -0.0112594718400889, 0.000...",[-1.23906359 1.27265975 2.84610163 0.605207...,3,[-9.10949224e-03 2.24824059e-02 -2.29044684e-...
2,AH017,18964,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936577870891, -0.0112608471333296, 0.000...",[-1.22969731 1.25002269 2.86874875 0.650720...,AH017,18964,Real,3604,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999935762540402, -0.0113340361155873, 0.000...",[-1.25467814 1.27784521 2.86638462 0.631967...,4,[-0.02466577 0.02807341 -0.00268453 -0.018804...
3,AH017,18964,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936577870891, -0.0112608471333296, 0.000...",[-1.22969731 1.25002269 2.86874875 0.650720...,AH017,18964,Real,3625,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999921037507743, -0.0125621447005781, 0.000...",[-1.35217096 1.32210321 3.00461354 0.807269...,25,[-0.12166161 0.07499366 0.13501497 0.156635...
4,AH017,18964,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999936577870891, -0.0112608471333296, 0.000...",[-1.22969731 1.25002269 2.86874875 0.650720...,AH017,18964,Real,3629,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.999923254983603, -0.012388825476446, 3.3857...",[-1.35123504e+00 1.29159350e+00 3.02518300e+...,29,[-0.12107048 0.04471094 0.15592978 0.142973...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3754,TM628,15932,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.997914, -0.044309, -0.046952, -5.31976, 0.0...",[-5.31976 8.77705 15.443406 6.762265...,TM628,15932,Real,5362,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.998191, -0.060127, 0.000249, -8.348157, 0.0...",[-8.348157 7.475679 12.826865 4.671239...,1762,[-3.20776001 -1.45956307 -2.30021811 -1.971353...
3755,TM628,15932,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.997914, -0.044309, -0.046952, -5.31976, 0.0...",[-5.31976 8.77705 15.443406 6.762265...,TM628,15932,Real,5366,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.998004, -0.062887, -0.005784, -8.481422, 0....",[-8.481422 5.10104 6.267603 2.034410...,1766,[-3.7720197 -4.56806524 -8.52110373 -4.623144...
3756,TM628,15932,Real,3600,/data16/public/registration-brain_PET/data/rea...,/data16/public/registration-brain_PET/data/rea...,"[0.997914, -0.044309, -0.046952, -5.31976, 0.0...",[-5.31976 8.77705 15.443406 6.762265...,TM628,15932,Real,5372,/data16/public/re

In [14]:
def delete_space(df):
    T_list=df['T']
    new_T=[]
    for T_el in T_list:
        T_el=T_el.replace('[','')
        T_el=T_el.replace(']','')
        T_tmp=[]
        string_split=T_el.split(' ')
        for i in range(len(string_split)):
            if string_split[i] != '' and string_split[i] != ' ':
                T_tmp.append(float(string_split[i]))
        new_T.append(np.array(T_tmp))

    df['T']=new_T
    return df

In [15]:
df_train=delete_space(df_train).reset_index(drop=True)
df_val=delete_space(df_val).reset_index(drop=True)
df_test=delete_space(df_test).reset_index(drop=True)

In [16]:
#Add the reference-moving time pairs (to use absolute time)
pairs=[]

for i in range(len(df_train)):
    pairs.append(np.array([df_train['ScanStart_ref'][i], df_train['ScanStart_mov'][i]]))
df_train['pairs']=pairs

pairs=[]

for i in range(len(df_val)):
    pairs.append(np.array([df_val['ScanStart_ref'][i], df_val['ScanStart_mov'][i]]))
df_val['pairs']=pairs


pairs=[]

for i in range(len(df_test)):
    pairs.append(np.array([df_test['ScanStart_ref'][i], df_test['ScanStart_mov'][i]]))
df_test['pairs']=pairs

In [17]:
# Building transform
KEYS = ['ThreeD_Cloud_ref','ThreeD_Cloud_mov'] 

train_transforms = Compose([
    LoadImaged(keys=KEYS, reader='NibabelReader', as_closest_canonical=False),
    monai.transforms.EnsureChannelFirstd(keys=KEYS), # previously AddChanneld
    Orientationd(keys=KEYS, axcodes='RAS'),
    ToTensord(keys=KEYS)    
])

In [ ]:
# Building training dataloader

tr_dict = df_train.to_dict('records')

for i in range(len(tr_dict)):
    x = tr_dict[i]['ThreeD_Cloud_ref'].find('nii')
    fn_cloud1 = tr_dict[i]['ThreeD_Cloud_ref'][0:x] + 'nii_monai_resize'
    x = x+3
    y = tr_dict[i]['ThreeD_Cloud_ref'].find('3dcld')
    fn_cloud2 =  tr_dict[i]['ThreeD_Cloud_ref'][x:y] + '3dcld_monai_rz.nii'
    tr_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2
    
    x = tr_dict[i]['ThreeD_Cloud_mov'].find('nii')
    fn_cloud1 = tr_dict[i]['ThreeD_Cloud_mov'][0:x] + 'nii_monai_resize'
    x = x+3
    y = tr_dict[i]['ThreeD_Cloud_mov'].find('3dcld')
    fn_cloud2 =  tr_dict[i]['ThreeD_Cloud_mov'][x:y] + '3dcld_monai_rz.nii'
    tr_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2

# Create the Dataset
ds_tr = monai.data.SmartCacheDataset(data=tr_dict,transform=train_transforms,replace_rate=0.25,cache_num=512,shuffle=True)
# Create the DataLoader
tr_loader = monai.data.DataLoader(ds_tr, batch_size=32, num_workers=4, collate_fn=list_data_collate)

In [ ]:
# Building validation dataloader

val_dict = df_val.to_dict('records')

for i in range(len(val_dict)):
    x = val_dict[i]['ThreeD_Cloud_ref'].find('nii')
    fn_cloud1 = val_dict[i]['ThreeD_Cloud_ref'][0:x] + 'nii_monai_resize'
    x = x+3
    y = val_dict[i]['ThreeD_Cloud_ref'].find('3dcld')
    fn_cloud2 =  val_dict[i]['ThreeD_Cloud_ref'][x:y] + '3dcld_monai_rz.nii'
    val_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2

    x = val_dict[i]['ThreeD_Cloud_mov'].find('nii')
    fn_cloud1 = val_dict[i]['ThreeD_Cloud_mov'][0:x] + 'nii_monai_resize'
    x = x+3
    y = val_dict[i]['ThreeD_Cloud_mov'].find('3dcld')
    fn_cloud2 =  val_dict[i]['ThreeD_Cloud_mov'][x:y] + '3dcld_monai_rz.nii'
    val_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2
    
# Create the Dataset
ds_val = monai.data.CacheDataset(data=val_dict, transform=train_transforms)
# Create the DataLoader
val_loader = monai.data.DataLoader(ds_val, batch_size=32, num_workers=4, collate_fn=list_data_collate)

In [ ]:
# Create a SmartCacheDataset Callback
class SmartCacheDatasetUpdate(pl.callbacks.Callback): # previously pl.callbacks.base.Callback

    def __init__(self, dataset):
        super().__init__()
                
        self.dataset = dataset
    
    def on_fit_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"): #removed the Nones
        """Called when fit begins."""
        print('Starting SmartCacheDataset')
        self.dataset.start()

    def on_fit_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):
        """Called when fit ends."""
        print('Stopping SmartCacheDataset')
        self.dataset.shutdown()

    def on_train_epoch_start(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):
        """Called when the train epoch start."""
        #print('Updating SmartCacheDataset')
        self.dataset.update_cache()
        

## Build network

In [ ]:
class PETRegNet(pl.LightningModule):
    
    def __init__(self, dropout=0.3):
        super().__init__()
        
        self.dropout = dropout
        
        self.feature_extractor = monai.networks.nets.resnet10(
                    spatial_dims=3, 
                    n_input_channels=1, 
                   num_classes=128
              )

        self.fwt_layers = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(32, 64),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(64, 128*2),            
        )
        
        self.regression_layers = torch.nn.Sequential(
            torch.nn.Linear(128*2, 256),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(256, 128),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(128, 6),
        )
        
        self.feature_extractor.apply(self._init_weights)
        self.fwt_layers.apply(self._init_weights)
        self.regression_layers.apply(self._init_weights)
        
        self.loss_function = torch.nn.MSELoss()
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if "conv" in classname.lower():
            if hasattr(m, 'weight'):
                torch.nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    torch.nn.init.xavier_normal_(m.weight)
        elif "linear" in classname.lower():
            if hasattr(m, 'weight'):
                torch.nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    torch.nn.init.xavier_normal_(m.weight)
        
    def forward(self, x1, x2, x_t):
        y1 = self.feature_extractor(x1)
        y2 = self.feature_extractor(x2)
        y = torch.cat([y1,y2], dim=-1)
        x_t=x_t.float()
        x_fwt = self.fwt_layers(x_t) 
        x_fwt=torch.flatten(x_fwt, start_dim=1)
        y = torch.mul(y, x_fwt)
        y = self.regression_layers(y)
        return y

    def prepare_data(self):
        # set deterministic training for reproducibility
        monai.utils.misc.set_determinism(seed=42)

    def training_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        xt= batch["pairs"]
        transforms = batch["T"].float()
        outputs = self.forward(x1,x2,xt)
        loss = self.loss_function(outputs, transforms)        
        return {"loss": loss}
        
    
    def training_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Train/Loss', avg_loss, self.current_epoch)
    
    
    def validation_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        xt= batch["pairs"]
        transforms = batch["T"].float()
        outputs = self.forward(x1,x2,xt)
        loss = self.loss_function(outputs, transforms)
        return {"val_loss": loss}

    
    def validation_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Val/Loss', avg_loss, self.current_epoch)

        # Log the value for model checkpoint saving
        self.log('val_loss', avg_loss.item()) #added .item(), otherwise validation wouldn't work 

    def configure_optimizers(self):
        total_params = list(self.feature_extractor.parameters()) + list(self.regression_layers.parameters()) + list(self.fwt_layers.parameters())
        opt = torch.optim.Adam(total_params, lr=5e-4)
        scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer=opt, step_size=1000, gamma=0.98),
                     'name': 'Learning Rate'}
        return [opt], [scheduler]

In [ ]:
from torchinfo import summary
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = PETRegNet().to(device)
summary(model, [(1,1,96,96,64),(1,1,96,96,64),(1,2)])

In [ ]:
MODEL_PATH = os.path.join('.','saved_model_1sub')
print('MODEL_PATH={}'.format(MODEL_PATH))

## Training

In [ ]:
# Initialise the LightningModule
model = PETRegNet(dropout=0.3)

# Set up loggers and checkpoints
tb_logger = pl.loggers.TensorBoardLogger(
    save_dir=os.path.join(MODEL_PATH,'logs')
)

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(MODEL_PATH),
    filename="PETRegNet-{epoch}-{val_loss:.3f}",
    monitor='val_loss',
    save_last=True,
    save_top_k=2,
    mode='min'
)

# Create and add the dataset callback
dataset_callback = SmartCacheDatasetUpdate(dataset=ds_tr)

# Check for last checkpoint
resume_checkpoint = None
if os.path.exists(os.path.join(MODEL_PATH,'last.ckpt')):
    resume_checkpoint = os.path.join(MODEL_PATH,'last.ckpt')

# Initialise Lightning's trainer.
trainer = pl.Trainer(
    accelerator='gpu',
    devices=[2],
#     gpus=[2], # deprecated since v1.7
    max_epochs=50, #10k is generally enough for single-subjct studies
    logger=tb_logger,
    callbacks=[dataset_callback,lr_monitor,checkpoint_callback],
    num_sanity_val_steps=1,
    check_val_every_n_epoch=10,
#     resume_from_checkpoint=resume_checkpoint # deprecated since v1.5
)

# Train
trainer.fit(model, tr_loader, val_loader, ckpt_path=resume_checkpoint) # previously was trainer.fit(model, train_dataloader=tr_loader, val_dataloaders=val_loader)

## Testing and evaluation

In [ ]:
from evaluation_toolbox import build_df_results, show_df_loss, plot_vicra_network, plot_diff_vicra_network, save_synthetic_vicra, print_loss

### Testing on testing data

In [ ]:
y_list_all_test=[]
predictions_all_test=[]

##building testing dataloader
test_dict = df_test.to_dict('records')

for i in range(len(test_dict)):
    x = test_dict[i]['ThreeD_Cloud_ref'].find('nii')
    fn_cloud1 = test_dict[i]['ThreeD_Cloud_ref'][0:x] + 'nii_monai_resize'
    x = x+3
    y = test_dict[i]['ThreeD_Cloud_ref'].find('3dcld')
    fn_cloud2 =  test_dict[i]['ThreeD_Cloud_ref'][x:y] + '3dcld_monai_rz.nii'
    test_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2

    x = test_dict[i]['ThreeD_Cloud_mov'].find('nii')
    fn_cloud1 = test_dict[i]['ThreeD_Cloud_mov'][0:x] + 'nii_monai_resize'
    x = x+3
    y = test_dict[i]['ThreeD_Cloud_mov'].find('3dcld')
    fn_cloud2 =  test_dict[i]['ThreeD_Cloud_mov'][x:y] + '3dcld_monai_rz.nii'
    test_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2

# Create the Dataset
ds_test = monai.data.CacheDataset(data=test_dict, transform=train_transforms)
#ds_tr = monai.data.SmartCacheDataset(data=tr_dict,transform=train_transforms,replace_rate=1,cache_num=64,shuffle=True)
# Create the DataLoader
test_loader = monai.data.DataLoader(ds_test, batch_size=1, num_workers=2, collate_fn=list_data_collate)

#calculate loss function and network output
saved_model_path = os.path.join(MODEL_PATH,'last.ckpt')
loaded_model = PETRegNet.load_from_checkpoint(saved_model_path)

loaded_model.eval()
loaded_model.to(device)
loss = 0
loss_list = list()
time_list = list()
prediction_list = list()
y_list = list()

i=0
for test_data in test_loader:
    x1 = test_data['ThreeD_Cloud_ref'].to(device)
    x2 = test_data['ThreeD_Cloud_mov'].to(device)
    x_t = test_data['pairs'].to(device)
    time = test_data['delta_t']
    y = test_data['T'].cpu().numpy()
    y_test = loaded_model(x1, x2, x_t).detach().cpu().numpy()

    loss = np.mean(np.square(y-y_test))
    i = i + 1
    loss_list.append(loss)
    time_list.append(time.numpy()[0])
    prediction_list.append(y_test)
    y_list.append(y)
    predictions_all_test.append(prediction_list)

y_list_all_test.append(y_list)

df_loss = pd.DataFrame()
df_loss['Time'] = time_list
df_loss['Loss'] = loss_list

In [ ]:
df_loss.describe()

In [ ]:
plt.plot(figsize=(8,5))
plt.title("Test loss")
#     plt.xlabel('Time(s)')
plt.ylabel('Loss (RMS)')
df_loss['Loss'].plot()
plt.show()

### Testing on testing patients (for multisubject studies)

In [ ]:
summaries_test, delta_T_all_test=compute_delta_T(['FDG'], test_set)
df_test=[]
predictions=[]
df_input_diff_all=[]

for i in range(len(test_set)):
    df_test.append(deal_dataframe(summaries_test[i]))

In [ ]:
len(df_test)

In [ ]:
predictions=[]
df_loss_all=[]

from copy import deepcopy

y_list_all=[]
df_input_diff_all=[]

for j in range(len(test_set)):
    
    df_loss=pd.DataFrame()

    df = df_test[j]

    #fixed reference time
    df_input_diff = vicra_toolbox.build_netinput_fixed_reference(df).reset_index()
    
    pairs=[]

    for i in range(len(df_input_diff)):
        pairs.append(np.array([df_input_diff['ScanStart_ref'][i], df_input_diff['ScanStart_mov'][i]]))
    df_input_diff['pairs']=pairs
    
    df_input_diff_all.append(df_input_diff)

    ##building testing dataloader
    test_dict = df_input_diff.to_dict('records')

    for i in range(len(test_dict)):
        x = test_dict[i]['ThreeD_Cloud_ref'].find('nii')
        fn_cloud1 = test_dict[i]['ThreeD_Cloud_ref'][0:x] + 'nii_monai_resize'
        x = x+3
        y = test_dict[i]['ThreeD_Cloud_ref'].find('3dcld')
        fn_cloud2 =  test_dict[i]['ThreeD_Cloud_ref'][x:y] + '3dcld_monai_rz.nii'
        test_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2

        x = test_dict[i]['ThreeD_Cloud_mov'].find('nii')
        fn_cloud1 = test_dict[i]['ThreeD_Cloud_mov'][0:x] + 'nii_monai_resize'
        x = x+3
        y = test_dict[i]['ThreeD_Cloud_mov'].find('3dcld')
        fn_cloud2 =  test_dict[i]['ThreeD_Cloud_mov'][x:y] + '3dcld_monai_rz.nii'
        test_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2

    # Create the Dataset
#     ds_test = monai.data.CacheDataset(data=test_dict, transform=train_transforms)
    ds_test = monai.data.Dataset(data=test_dict, transform=train_transforms)
    #ds_tr = monai.data.SmartCacheDataset(data=tr_dict,transform=train_transforms,replace_rate=1,cache_num=64,shuffle=True)
    # Create the DataLoader
    test_loader = monai.data.DataLoader(ds_test, batch_size=1, num_workers=2, collate_fn=list_data_collate)
    
    #calculate loss function and network output
    saved_model_path = os.path.join(MODEL_PATH,'last.ckpt')
    loaded_model = PETRegNet.load_from_checkpoint(saved_model_path)

    loaded_model.eval()
    loaded_model.to(device)
    loss = 0
    loss_list = list()
    time_list = list()
    prediction_list = list()
    y_list = list()

    i=0
    for test_data in test_loader:
        x1 = test_data['ThreeD_Cloud_ref'].to(device)
        x2 = test_data['ThreeD_Cloud_mov'].to(device)
        x_t = test_data['pairs'].to(device)
        time = test_data['delta_t']
        y = test_data['T'].cpu().numpy()
        y_test = loaded_model(x1, x2, x_t).detach().cpu().numpy()

        loss = np.mean(np.square(y-y_test))
        i = i + 1
        loss_list.append(loss)
        time_list.append(time.numpy()[0])
        prediction_list.append(y_test)
        y_list.append(y)
        
        del test_data

    # loss = loss / 500
    df_loss['Loss']=loss_list
    df_loss_all.append(df_loss)
    df_results = pd.DataFrame()
    predictions.append(prediction_list)
    y_list_all.append(y_list)

In [ ]:
df_loss_all[0].describe()

In [ ]:
# Flatten predictions
predictions_flat=[]
for predi in predictions:
    for element in predi:
        predictions_flat.append(element[0])

In [ ]:
df_results_test = build_df_results(test_set, df_input_diff_all, predictions_flat)

In [ ]:
df_results_test

In [ ]:
# df_results_test.to_csv('results/DL-HMC_original_mulsub21.csv', index=False)

In [ ]:
plot_vicra_network('FDG', 'AF120', df_results_test, [3601,5399], 'ResNet')

In [ ]:
plot_diff_vicra_network('FDG', 'DS636', df_results_test, [3601,5399], 'ResNet')

In [ ]:
# save_synthetic_vicra('FDG', 'AO101', df_results_test, [3601,5399], 'test')

In [ ]:
from evaluation_toolbox import plot_networks_comparison
plot_networks_comparison('FDG', 'JO308', df_results_test, [3601, 5399], 'synthetic_vicra_test_AO101.csv', 'ResNet', 'DenseNet') # Mixing these two patients doesn't really make sense, it's just a test

In [ ]:
# Make boxplot to compare inference RMSE from different models

from evaluation_toolbox import make_boxplot

df_other_model=pd.read_csv('2d_mean_results', sep=',') # Load the df_results csv file from another training (this file has nothing to do with this study, it's just an example)

def delete_space_v(df):
    T_list=df['vicra parameters']
    new_T=[]
    for T_el in T_list:
        T_el=T_el.replace('[','')
        T_el=T_el.replace(']','')
        T_tmp=[]
        string_split=T_el.split(' ')
        for i in range(len(string_split)):
            if string_split[i] != '' and string_split[i] != ' ':
                T_tmp.append(float(string_split[i]))
        new_T.append(np.array(T_tmp))

    df['vicra parameters']=new_T
    return df

def delete_space_m(df):
    T_list=df['model parameters']
    new_T=[]
    for T_el in T_list:
        T_el=T_el.replace('[','')
        T_el=T_el.replace(']','')
        T_tmp=[]
        string_split=T_el.split(' ')
        for i in range(len(string_split)):
            if string_split[i] != '' and string_split[i] != ' ':
                T_tmp.append(float(string_split[i]))
        new_T.append(np.array(T_tmp))

    df['model parameters']=new_T
    return df

df_other_model=delete_space_v(df_other_model)
df_other_model=delete_space_m(df_other_model)

make_boxplot(df_results_test, df_other_model, models=['model1', 'model2'])